### PyEmma Featurizer Support

In [1]:
import openpathsampling as ps
import openpathsampling.storage as st

In [2]:
#! lazy
import pyemma.coordinates as coor

In [3]:
storage = st.Storage('trajectory.nc', mode='a')

Import a PyEmma Coordinates Module

Using of pyemma featurizers or general other complex code requires a little trick to be storable. Since storing of code only works if we are not dependend on the context (scope) we need to wrap the construction of our featurizer in a function, that gets all it needs from the global scope as a parameter

In [4]:
def pyemma_generator(f):
    f.add_inverse_distances(f.pairs(f.select_Backbone()))

In [5]:
cv = ps.collectivevariable.CV_PyEMMA_Featurizer('pyemma', pyemma_generator, topology=storage.template.topology)

Now use this featurizer generating function to build a collective variable out of it. All we need for that is a name as usual, the generating function, the list of parameters - here only the topology and at best a test snapshot, a template.

In [6]:
cv(storage.trajectories[0]);

Let's save it to the storage

In [7]:
cv.save(storage)

and apply the featurizer to a trajectory

In [8]:
cv(storage.trajectories[2]);

Sync to make sure the cache is written to the netCDF file.

In [9]:
storage.sync_all()

In [10]:
cv(storage.snapshots.all());

In [11]:
storage.cvs.sync()

In [12]:
storage.sync_all()

In [13]:
py_cv = storage.cvs['pyemma']

In [14]:
nc_var = storage.variables['snapshots_cv_%d_values' % storage.idx(py_cv)]

In [15]:
assert(nc_var.shape[1] == 15)
print nc_var.shape[1]

15


In [16]:
assert(nc_var.var_type == 'numpy.float32')
print nc_var.var_type

numpy.float32


In [17]:
#! ignore
print storage.variables['cvs_json'][:]

[ u'{"_cls": "CV_MDTraj_Function", "_dict": {"store_cache": true, "name": "psi", "f": {"_module": "mdtraj.geometry.dihedral", "_name": "compute_dihedrals"}, "scalarize_numpy_singletons": true, "wrap_numpy_array": true, "kwargs": {"indices": [[6, 8, 14, 16]]}, "requires_lists": true, "time_reversible": true}}'
 u'{"_cls": "CV_MDTraj_Function", "_dict": {"store_cache": true, "name": "phi", "f": {"_module": "mdtraj.geometry.dihedral", "_name": "compute_dihedrals"}, "scalarize_numpy_singletons": true, "wrap_numpy_array": true, "kwargs": {"indices": [[4, 6, 8, 14]]}, "requires_lists": true, "time_reversible": true}}'
 u'{"_cls": "CV_Volume", "_dict": {"volume": {"_obj": "volumes", "_idx": 0}, "store_cache": true, "name": "StateA"}}'
 u'{"_cls": "CV_Volume", "_dict": {"volume": {"_obj": "volumes", "_idx": 25}, "store_cache": true, "name": "StateB"}}'
 u'{"_cls": "CV_Volume", "_dict": {"volume": {"_obj": "volumes", "_idx": 26}, "store_cache": true, "name": "StateX"}}'
 u'{"_cls": "CV_PyEMMA_F

In [18]:
py_emma_feat = storage.vars['cvs_json'][5]

In [19]:
erg = py_emma_feat(storage.snapshots);

In [20]:
#! lazy
print erg[0::10,1]

[ 4.05474138  4.03769112  4.06672716  4.09396172  3.97718644  4.03076649
  4.04023886  3.96421409  3.98849988  4.16449261  4.12263393  3.96812487
  3.97319508  3.92330384  4.17456722  4.05569458  3.94226432  4.04616547
  3.93947411  4.01203823  3.97155762  3.9882586   3.91708589  4.08536196
  3.91899133  4.00646782  4.07670736  3.95574212  4.05487919  4.06668091
  3.96205282  3.91439772  4.01757526  3.90480232  3.98151636  3.88471007
  4.1294179   3.90275097  3.98615885  4.03889561  4.09112215  3.89952517
  3.99120545  3.9772594   4.1503377   4.04966497  4.05266047  4.07484865
  3.95554161  4.02089167  3.96684408  4.06132174  3.94756866  4.11182117]
